In [1]:
import pandas as pd 
import numpy as np
import regex as re
import nltk
# !pip install gensim
# !pip install spacy
import spacy
from spacy import lemmatizer
import gensim
import regex
my_punctuation = '!"$%&\'()*+,-./:;<=>?[\\]^_`{|}~•@'
my_stopwords = nltk.corpus.stopwords.words('english')
print(my_stopwords)
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
lemma = WordNetLemmatizer()
import json

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [15]:
ingredient_df=pd.read_csv('../Dataset/ingredientsv1.csv')
amazon_food_review_df=pd.read_csv('../Dataset/Reviews.csv')
print(ingredient_df)
print(amazon_food_review_df)

                        id                  asins          brand  \
0     AVphBRHOilAPnD_x0OrE             B00HXST15C  Simon Fischer   
1     AVpfNFy1LJeJML434ma2  B008VT0W8C,B0092F8OJ8      McCormick   
2     AVpgT49VLJeJML43MJEz             B00CHTWZ2S     Jolly Time   
3     AVphYgnzLJeJML43aPp2             B002JJYNVW          Ziyad   
4     AVpiS0bOLJeJML43kRsh             B00290W1CY    Fla-Vor-Ice   
...                    ...                    ...            ...   
9995  AVpfVL3ailAPnD_xZyHc  B00BHNV8N2,B06XC7GXCD       Wright's   
9996  AVpgK5BZilAPnD_xofxu             B00HZO4YYW     Eden Foods   
9997  AV00srHj-jtxr-f30m1M  B01FRJ0XKA,B01IV1D1VY    Happy Belly   
9998  AVpfmY4RLJeJML43ANr-  B000PWIR7Q,B005ER1C6U         Quaker   
9999  AVpe-j33LJeJML43zpy-             B001EO5JUC       Weetabix   

                                             categories             dateAdded  \
0                   Grocery & Gourmet Food,Food,Grocery  2017-01-07T20:13:17Z   
1                   G

In [17]:
# print(amazon_food_review_df.head(1))
# print(len(amazon_food_review_df['Summary'].unique()))
food_review_text=amazon_food_review_df['Text']
print(food_review_text)

0         I have bought several of the Vitality canned d...
1         Product arrived labeled as Jumbo Salted Peanut...
2         This is a confection that has been around a fe...
3         If you are looking for the secret ingredient i...
4         Great taffy at a great price.  There was a wid...
                                ...                        
568449    Great for sesame chicken..this is a good if no...
568450    I'm disappointed with the flavor. The chocolat...
568451    These stars are small, so you can give 10-15 o...
568452    These are the BEST treats for training and rew...
568453    I am very satisfied ,product is as advertised,...
Name: Text, Length: 568454, dtype: object


In [23]:
# doc_clean_list=[]
def clean_doc(text):
    stop_free=' '.join([i for i in text.lower().split() if i not in my_stopwords])
    punc_free = ''.join(ch for ch in stop_free if ch not in my_punctuation)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

doc_clean = [clean_doc(doc).split() for doc in food_review_text]  
# with open('preprocessed.txt', 'w') as f:
#     for i in doc_clean:
#         f.write(str(i)+'\n')

In [24]:
amazon_food_review_df['text_token_list']=doc_clean
print(amazon_food_review_df['text_token_list'])

0         [bought, several, vitality, canned, dog, food,...
1         [product, arrived, labeled, jumbo, salted, pea...
2         [confection, around, century, light, pillowy, ...
3         [looking, secret, ingredient, robitussin, beli...
4         [great, taffy, great, price, wide, assortment,...
                                ...                        
568449    [great, sesame, chickenthis, good, better, res...
568450    [im, disappointed, flavor, chocolate, note, es...
568451    [star, small, give, 1015, one, training, sessi...
568452    [best, treat, training, rewarding, dog, good, ...
568453    [satisfied, product, advertised, use, cereal, ...
Name: text_token_list, Length: 568454, dtype: object


In [26]:
amazon_food_review_df.head(5)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,text_token_list
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...,"[bought, several, vitality, canned, dog, food,..."
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,"[product, arrived, labeled, jumbo, salted, pea..."
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...,"[confection, around, century, light, pillowy, ..."
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...,"[looking, secret, ingredient, robitussin, beli..."
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...,"[great, taffy, great, price, wide, assortment,..."


In [28]:
amazon_food_review_df['Score'].unique()

array([5, 1, 4, 2, 3], dtype=int64)

In [66]:
doc1 = "Sugar is bad to consume. My sister likes to have sugar, but not my father."
doc2 = "My father spends a lot of time driving my sister around to dance practice."
doc3 = "Doctors suggest that driving may cause increased stress and blood pressure."
doc4 = "Sometimes I feel pressure to perform well at school, but my father never seems to drive my sister to do better."
doc5 = "Health experts say that Sugar is not good for your lifestyle."

# compile documents
doc_complete = [doc1, doc2, doc3, doc4, doc5]

from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

In [67]:
doc_clean = [clean(doc).split() for doc in doc_complete]   

In [68]:
print(doc_clean)

[['sugar', 'bad', 'consume', 'sister', 'like', 'sugar', 'father'], ['father', 'spends', 'lot', 'time', 'driving', 'sister', 'around', 'dance', 'practice'], ['doctor', 'suggest', 'driving', 'may', 'cause', 'increased', 'stress', 'blood', 'pressure'], ['sometimes', 'feel', 'pressure', 'perform', 'well', 'school', 'father', 'never', 'seems', 'drive', 'sister', 'better'], ['health', 'expert', 'say', 'sugar', 'good', 'lifestyle']]
